## DMRG using MPS: Hubbard model

This program does a DMRG calculation of the ground state of the fermionic Hubbard model by variationally optimizing an MPS. The code for doing tensor operations and sweep iterations has been adapted from a tutorial by Ian McCulloch (https://people.smp.uq.edu.au/IanMcCulloch/mptoolkit/index.php?n=Tutorials.MPS-DMRG?from=APCTPWorkshop.MPS-DMRG) with minor changes and a few additional comments. It is not meant to be efficient, but it is very readable. Here, MPS and MPO are stored as lists of the size of the chain, with each member being a local tensor. 

In [27]:
    #####################################################
    # Simple DMRG code using MPS/MPO representations    #
    # Ian McCulloch August 2017 (slightly modified)     #
    #####################################################
     
    import numpy as np
    import scipy
    import scipy.sparse.linalg
    import scipy.sparse as sparse
    import math
     
    #  3-index local tensor states A[s,i,j]
    #      s
    #      |
    #   i -A- j 
    #
    # [s] acts on the local Hilbert space, physical index
    # [i,j] act on the virtual bonds, virtual index 
    #
    #  MPS: contracted local tensor states
    #
    #       s0    s1   s2       s(L-2)    s(L-1)
    #       |     |    |          |         |
    #      A0 -- A1 -- A2--...--A(L-2) -- A(L-1)
    # 
    #  stored as [A0,A1,...,A(L-1)]
    #
    
    # 4-index local tensor operators W[s,t,i,j]
    #     s
    #     |
    #  i -W- j
    #     |
    #     t
    #
    # [s,t] act on the local Hilbert space, physical indices
    # [i,j] act on the virtual bonds, virtual indices
    #
    #  MPO: contracted local tensor operators
    #
    #       s0    s1   s2       s(L-2)    s(L-1)
    #       |     |    |          |         |
    #      W0 -- W1 -- W2--...--W(L-2) -- W(L-1)
    #      |     |     |          |         |
    #      t0    t1    t2       t(L-2)    t(L-1)
    #
    #  stored as [W0,W1,...,W(L-1)]
    # 
        
    
    # E and F are the environment tensors on the left (E) and right (F) side of the sites being optimized
    # we choose to start from the left hand side i.e. optimizing the 1-2 site block, so the initial E matrix
    # is one, the initial F matrices cover the rest of the chain
    ## tensor contraction from the right hand side
    ##  -+      -A--+
    ##   |       |  |
    ##  -F' =   -W--F
    ##   |       |  |
    ##  -+      -B--+
    def contract_from_right(W, A, F, B):
        # contracting efficiently in a sequence (starting at an edge and moving progressively along the network
        # is a good rule of thumb)
        # the comments show order of computational cost
        Temp = np.einsum("sij,bjl->sbil", A, F) # d m_s^3 m_o
        Temp = np.einsum("sbil,abst->tail", Temp, W) # d^2 m_s^2 m_o^2 
        return np.einsum("tail,tkl->aik", Temp, B) # d m_s^3 m_o 
     
    ## tensor contraction from the left hand side
    ## +-    +--A-
    ## |     |  |
    ## E' =  E--W-
    ## |     |  |
    ## +-    +--B-
    def contract_from_left(W, A, E, B):
        # contracting efficiently in a sequence
        Temp = np.einsum("sij,aik->sajk", A, E)
        Temp = np.einsum("sajk,abst->tbjk", Temp, W)
        return np.einsum("tbjk,tkl->bjl", Temp, B)
     
    # construct the initial E and F tensors. 
    # they are stored as they are calculated, because they can be used in subsequent sweeps.
    def initial_F(MPO, MPS):
        temp = np.zeros((MPO[-1].shape[1],1,1))
        temp[-1] = 1
        F = [temp]
     
        for i in range(len(MPO)-1, 1, -1):
            F.append(contract_from_right(MPO[i], MPS[i], F[-1], MPS[i]))
        return F
     
    def initial_E(MPO):
        E = np.zeros((MPO[0].shape[0],1,1))
        E[0] = 1
        return [E]
     
    # 2-1 coarse-graining of two site MPO into one site
    #  |     |  |
    # -R- = -W--X-
    #  |     |  |
    def coarse_grain_MPO(W, X):
        return np.reshape(np.einsum("abst,bcuv->acsutv",W,X),
                          [W.shape[0], X.shape[1],
                           W.shape[2]*X.shape[2],
                           W.shape[3]*X.shape[3]])
   
     
     
    # 2-1 coarse-graining of two-site MPS into one site
    #   |     |  |
    #  -R- = -A--B-
    def coarse_grain_MPS(A,B):
        return np.reshape(np.einsum("sij,tjk->stik",A,B),
                          [A.shape[0]*B.shape[0], A.shape[1], B.shape[2]])
     
    # opposite of coarse graining, decompose using SVD    
    def fine_grain_MPS(A, dims):
        assert A.shape[0] == dims[0] * dims[1]
        Theta = np.transpose(np.reshape(A, dims + [A.shape[1], A.shape[2]]),
                             (0,2,1,3))
        M = np.reshape(Theta, (dims[0]*A.shape[1], dims[1]*A.shape[2]))
        U, S, V = np.linalg.svd(M, full_matrices=0)
        U = np.reshape(U, (dims[0], A.shape[1], -1))
        V = np.transpose(np.reshape(V, (-1, dims[1], A.shape[2])), (1,0,2))
        return U, S, V
     
    # truncate the matrices from an SVD to at most m states
    def truncate_SVD(U, S, V, m):
        m = min(len(S), m)
        trunc = np.sum(S[m:])
        S = S[0:m]
        U = U[:,:,0:m]
        V = V[:,0:m,:]
        return U,S,V,trunc,m
     
    # Functor to evaluate the Hamiltonian matrix-vector multiply
    #        +--A--+
    #        |  |  |
    # -R- =  E--W--F
    #  |     |  |  |
    #        +-   -+
    class HamiltonianMultiply(sparse.linalg.LinearOperator):
        def __init__(self, E, W, F):
            self.E = E
            self.W = W
            self.F = F
            self.dtype = np.dtype('d')
            self.req_shape = [W.shape[2], E.shape[1], F.shape[2]]
            self.size = self.req_shape[0]*self.req_shape[1]*self.req_shape[2]
            self.shape = [self.size, self.size]
     
        def _matvec(self, A):
            # contracting efficiently
            R = np.einsum("aij,sik->ajsk", self.E, np.reshape(A, self.req_shape))
            R = np.einsum("ajsk,abst->bjtk", R, self.W)
            R = np.einsum("bjtk,bkl->tjl", R, self.F)
            return np.reshape(R, -1)

     
    ## two-site optimization of MPS A,B with respect to MPO W1,W2 and
    ## environment tensors E,F
    ## dir = 'left' or 'right' for a left-moving or right-moving sweep
    def optimize_two_sites(A, B, W1, W2, E, F, m, dir):
        W = coarse_grain_MPO(W1,W2)
        AA = coarse_grain_MPS(A,B)
        H = HamiltonianMultiply(E,W,F)
        Ene,V = sparse.linalg.eigsh(H,1,v0=AA,which='SA')
        AA = np.reshape(V[:,0], H.req_shape)
        A,S,B = fine_grain_MPS(AA, [A.shape[0], B.shape[0]])
        A,S,B,trunc,m = truncate_SVD(A,S,B,m)
        if (dir == 'right'):
            B = np.einsum("ij,sjk->sik", np.diag(S), B)
        else:
            assert dir == 'left'
            A = np.einsum("sij,jk->sik", A, np.diag(S))
        return Ene[0], A, B, trunc, m
     
        
    ## Driver function to perform sweeps of 2-site DMRG
    def two_site_dmrg(MPS, MPO, m, maxSweeps, tol):
        E = initial_E(MPO)
        F = initial_F(MPO, MPS)
        eneOld = 0
        for sweep in range(0,int(maxSweeps/2)):
            for i in range(0, len(MPS)-2):
                Energy,MPS[i],MPS[i+1],trunc,states = optimize_two_sites(MPS[i],MPS[i+1],
                                                                         MPO[i],MPO[i+1],
                                                                         E[-1], F[-1], m, 'right')
                print("Sweep {:} Sites {:},{:}    Energy {:16.12f}    States {:4} Truncation {:16.12f}"
                         .format(sweep*2,i,i+1, Energy, states, trunc))
                E.append(contract_from_left(MPO[i], MPS[i], E[-1], MPS[i]))
                F.pop();
            for i in range(len(MPS)-2, 0, -1):
                Energy,MPS[i],MPS[i+1],trunc,states = optimize_two_sites(MPS[i],MPS[i+1],
                                                                         MPO[i],MPO[i+1],
                                                                         E[-1], F[-1], m, 'left')
                print("Sweep {} Sites {},{}    Energy {:16.12f}    States {:4} Truncation {:16.12f}"
                         .format(sweep*2+1,i,i+1, Energy, states, trunc))
                F.append(contract_from_right(MPO[i+1], MPS[i+1], F[-1], MPS[i+1]))
                E.pop();
            if abs(eneOld-Energy) < tol:
                break
            else:
                eneOld = Energy
        return Energy, MPS

Now, we use these routines to find the ground state of the Hubbard model. The Hamiltonian is given as 
$$
H=t\sum_{i=1,\sigma}^{L-1}\left(a_{i\sigma}^{\dagger}a_{i+1\sigma}+\text{h.c.}\right)+U\sum_{i=1}^{L}n_{\uparrow}n_{\downarrow}.
$$
In this case, the local Hilbert space at site $i$ has a basis: $ \mid0\rangle_i, \mid\uparrow\rangle_i, \mid\downarrow\rangle_i, \mid\uparrow\downarrow\rangle_i $. We start with the initial state as the product state of singly occupied alternating spins: $ \mid \uparrow\downarrow\uparrow\dots\downarrow\uparrow\downarrow\rangle $. This state is variationally optimized iteratively using the two-site algorithm defined above. No symmetries are imposed on the system (not even particle number symmetry!). You can try using different values of $m$ and see how the ground state energy changes. 

# Note on MPO representation of fermionic operators 
As seen above, MPS formalism of DMRG is cast in terms of Fock spaces. One advantage of using Fock spaces is that it affords a product representation of composite spaces. For example, the Fock space at a single site $i$ of the Hubbard model is given by:  $\mathcal{F}_i=\text{Range}\{\mid0\rangle_i, \mid\uparrow\rangle_i, \mid\downarrow\rangle_i, \mid\uparrow\downarrow\rangle_i\}$. The Fock space of composition of two sites $i$ and $j$ is the product space of the two local Fock spaces on each site: $\mathcal{F}_{ij}=\mathcal{F}_i\otimes\mathcal{F}_j$. Note that this is different from a fixed particle number representation of fermionic spaces, where the composite space is an antisymmetrized product. In the Fock space formalism, the onus of antisymmetry is laid upon operators. This can be analyzed in a couple of different but equivalent ways. One of the ways is to look at the anticommutation relations between fermionic operators. Equivalently we can analyze the action of these operators on occupation number basis states. We will follow the second approach. The defining relations for creation operators are given by
$$
a^{\dagger}_{l\uparrow}\mid n_{1\uparrow}n_{1\downarrow}\dots n_{l\uparrow}n_{l\downarrow}\dots n_{L\uparrow}n_{L\downarrow}\rangle=(-1)^{\Gamma_{l\uparrow}}\delta_{n_{l\uparrow}0}\mid n_{1\uparrow}n_{1\downarrow}\dots 1n_{l\downarrow}\dots n_{L\uparrow}n_{L\downarrow}\rangle,
$$
$$
a^{\dagger}_{l\downarrow}\mid n_{1\uparrow}n_{1\downarrow}\dots n_{l\uparrow}n_{l\downarrow}\dots n_{L\uparrow}n_{L\downarrow}\rangle=(-1)^{\Gamma_{l\downarrow}}\delta_{n_{l\downarrow}0}\mid n_{1\uparrow}n_{1\downarrow}\dots n_{l\uparrow}1\dots n_{L\uparrow}n_{L\downarrow}\rangle,
$$
where $\Gamma_{l\uparrow}=\Pi_{i<l}(-1)^{n_i}$ and $\Gamma_{l\downarrow}=(-1)^{n_{l\uparrow}}\Pi_{i<l}(-1)^{n_i}$ are parity factors. Analogous relations hold for annihilation operators. Locally these can be represented by the matrices
$$
\left[a^{\dagger}_{l\uparrow}\right]=\begin{bmatrix} 
                                 0 & 0 & 0 & 0\\
                                 1 & 0 & 0 & 0\\
                                 0 & 0 & 0 & 0\\
                                 0 & 0 & 1 & 0\\
                                 \end{bmatrix},\quad
\left[a^{\dagger}_{l\downarrow}\right]=\begin{bmatrix} 
                                 0 & 0 & 0 & 0\\
                                 0 & 0 & 0 & 0\\
                                 1 & 0 & 0 & 0\\
                                 0 & -1 & 0 & 0\\
                                 \end{bmatrix}.
$$                             
To construct their representations on the whole Fock space we have to take into account the parity factors. The obvious first choice of $I_1\otimes\dots\otimes a^{\dagger}_{l\sigma}\otimes\dots\otimes I_L$ for $a^{\dagger}_{l\sigma}$ won't work, as it doesn't reproduce the parity factors required for anticommutation. The Jordan-Wigner transform, introduced to map bosonic spin systems to non-interacting fermions, can be used to solve the problem. Consider the operator $P_1\otimes P_2\otimes\dots\otimes P_{l-1}\otimes a^{\dagger}_{l\sigma}\otimes I_{l+1}\otimes\dots\otimes I_L$, where $P_l$ are parity operators given by
$$
\left[P_l\right]=\begin{bmatrix} 
                  1 & 0 & 0 & 0\\
                  0 & -1 & 0 & 0\\
                  0 & 0 & -1 & 0\\
                  0 & 0 & 0 & 1\\
                  \end{bmatrix}.
$$
The parity operators have negative diagonal values for single occupancy states. The string of parity operators used is called a Jordan-Wigner (JW) string. These strings can be used as above to represent operators on the whole space. 


When representing the Hubbard Hamiltonian as a MPO, we only need to worry about parity factors for the hopping term. Consider the term $a^{\dagger}_{l\sigma}a_{l+1\sigma}$. Using the JW strings we can see that this term has the global form: $I_1\otimes I_2 \otimes\dots\otimes a^{\dagger}_{l\sigma}P_l\otimes a_{l+1\sigma}\otimes I_{l+2}\otimes\dots I_L$. Here we have used the fact that $P_l^2=I_l$. Let's define
$$
\left[a^{\dagger}_{lp\uparrow}\right]=\left[a^{\dagger}_{l\uparrow}P_l\right]=\begin{bmatrix} 
                                 0 & 0 & 0 & 0\\
                                 1 & 0 & 0 & 0\\
                                 0 & 0 & 0 & 0\\
                                 0 & 0 & -1 & 0\\
                                 \end{bmatrix},\quad
\left[a^{\dagger}_{lp\downarrow}\right]=\left[a^{\dagger}_{l\downarrow}P_l\right]=\begin{bmatrix} 
                                 0 & 0 & 0 & 0\\
                                 0 & 0 & 0 & 0\\
                                 1 & 0 & 0 & 0\\
                                 0 & 1 & 0 & 0\\
                                 \end{bmatrix},
$$
where the $p$ stands for parity. 


Using these we can represent the Hubbard Hamiltonian as a MPO using the following matrices:
$$
W_1=\begin{bmatrix}
      I_1 & a^{\dagger}_{1\uparrow} & a^{\dagger}_{1\downarrow} & a_{1\uparrow} & a_{1\downarrow} & Un_{1}
      \end{bmatrix},\quad
W_l(l\neq 1,L)=\begin{bmatrix}
      I_l & a^{\dagger}_{l\uparrow} & a^{\dagger}_{l\downarrow} & a_{l\uparrow} & a_{l\downarrow} & Un_{l}\\
      0 & 0 & 0 & 0 & 0 &  ta_{lp\uparrow}\\
      0 & 0 & 0 & 0 & 0 &  ta_{lp\downarrow}\\
      0 & 0 & 0 & 0 & 0 & -ta^{\dagger}_{lp\uparrow}\\
      0 & 0 & 0 & 0 & 0 & -ta^{\dagger}_{lp\downarrow}\\
      0 & 0 & 0 & 0 & 0 &  I_l\\
    \end{bmatrix},\quad
W_L=\begin{bmatrix}
      Un_{L}\\
      ta_{Lp\uparrow}\\
      ta_{Lp\downarrow}\\
      -ta^{\dagger}_{Lp\uparrow}\\
      -ta^{\dagger}_{Lp\downarrow}\\
      I_L\\
    \end{bmatrix}.
$$
The Hamiltonian is then given by $H=\prod_{l=1}^L W_l$, as can be verified by explicitly multiplying a few terms. 

In [52]:
##
## Parameters for the DMRG simulation
##
 
d=4   # local bond dimension, |0>, |up>, |dn>, |up,dn>
N=6   # number of sites
t=-1  # hopping element
u=2   # on-site repulsion
## initial state: |up dn up dn up dn >
 
InitialA1 = np.zeros((d,1,1))
InitialA1[1,0,0] = 1
InitialA2 = np.zeros((d,1,1))
InitialA2[2,0,0] = 1
 
MPS = [InitialA1, InitialA2] * int(N/2)
 
## Local operators
I = np.identity(4)
Z = np.zeros((4,4))
aDagUp = np.array([[0, 0, 0, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0]])
aDagUpPar = np.array([[0, 0, 0, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, -1, 0]])
aDagDn = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [1, 0, 0, 0],
              [0, -1, 0, 0]])
aDagDnPar = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [1, 0, 0, 0],
              [0, 1, 0, 0]])
aUp = np.array([[0, 1, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 1],
              [0, 0, 0, 0]])
aUpPar = np.array([[0, -1, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 1],
              [0, 0, 0, 0]])
aDn = np.array([[0, 0, 1, 0],
              [0, 0, 0, -1],
              [0, 0, 0, 0],
              [0, 0, 0, 0]])
aDnPar = np.array([[0, 0, -1, 0],
              [0, 0, 0, -1],
              [0, 0, 0, 0],
              [0, 0, 0, 0]])
nUpnDn = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 1]])

 
## Hamiltonian MPO
W = np.array([[I, aDagUp, aDagDn, aUp, aDn, u*nUpnDn],
              [Z,   Z,      Z,     Z,   Z,    t*aUpPar ],
              [Z,   Z,      Z,     Z,   Z,    t*aDnPar ],
              [Z,   Z,      Z,     Z,   Z, -t*aDagUpPar],
              [Z,   Z,      Z,     Z,   Z, -t*aDagDnPar],
              [Z,   Z,      Z,     Z,   Z,      I   ]])
 
# left-hand edge is 1x5 matrix
Wfirst = np.array([[I, aDagUp, aDagDn, aUp, aDn, u*nUpnDn]])
 
# right-hand edge is 5x1 matrix
Wlast = np.array([[u*nUpnDn], [t*aUpPar], [t*aDnPar], [-t*aDagUpPar], [-t*aDagDnPar],[I]])
 
# the complete MPO
MPO = [Wfirst] + ([W] * (N-2)) + [Wlast]
 

# sweep parameters
m = 50
maxSweeps = 8
conv_tol = 0.0001
Energy, MPS = two_site_dmrg(MPS, MPO, m, maxSweeps, conv_tol)
print("Final energy expectation value {}".format(Energy))


Sweep 0 Sites 0,1    Energy  -1.236067977500    States    4 Truncation   0.000000000000
Sweep 0 Sites 1,2    Energy  -2.279452315769    States    4 Truncation   0.000000000000
Sweep 0 Sites 2,3    Energy  -2.947763580272    States    4 Truncation   0.000000000000
Sweep 0 Sites 3,4    Energy  -3.844762445028    States    4 Truncation   0.000000000000
Sweep 1 Sites 4,5    Energy  -4.781748639473    States    4 Truncation   0.000000000000
Sweep 1 Sites 3,4    Energy  -4.819493953859    States   16 Truncation   0.000000000000
Sweep 1 Sites 2,3    Energy  -4.990934146792    States   16 Truncation   0.000000000000
Sweep 1 Sites 1,2    Energy  -5.017453229281    States   16 Truncation   0.000000000000
Sweep 2 Sites 0,1    Energy  -5.017453229281    States    4 Truncation   0.000000000000
Sweep 2 Sites 1,2    Energy  -5.017453229281    States   16 Truncation   0.000000000000
Sweep 2 Sites 2,3    Energy  -5.017468463489    States   50 Truncation   0.000000000000
Sweep 2 Sites 3,4    Energy  -5.

# References

1. Schollwöck, U. (2011). The density-matrix renormalization group in the age of matrix product states. Annals of Physics, 326(1), 96-192. 
2. Hubig, C., McCulloch, I. P., & Schollwöck, U. (2017). Generic construction of efficient matrix product operators. Physical Review B, 95(3), 035129.
3. http://tqm.courses.phy.cam.ac.uk/docs/lectures/JordanWigner/